# `Calibration-pixelmatch.ipynb`: Simulation-based calibration for ndlar-sim for hits (voxel-match version).
Attempts to determine the conversion factor from "electrons at anode" to MeV by matching voxels in a pair of files---one that was voxelized directly from edep-sim, and another that was run through larnd-sim and then subsequently voxelized onto the same grid.

This apes a lot of the functionality of the Plots.py chain, but it's implemented separately since that toolkit doesn't have any support for comparisons of two files that are have "the same" events in them, and implementing it would be a nightmare.

Unfortunately it turns out that this method doesn't work.  I believe the main reason is the double-discretization that happens for the larnd-sim hits: first when mapped onto the anode pads, and then again when forced onto the grid in the reconstruction.  The result is that energy often is shifted to a neighboring voxel due to this rasterization (confirmed by examining event displays) in a difficult-to-undo way.
The attempt is kept here for posterity...

original author: J. Wolcott <jwolcott@fnal.gov>
date:            March 2022

In [ ]:
import sys
sys.path.append("/media/disk1/jwolcott/app/dune-nd-lar-reco")

In [ ]:
# need an edep-sim and larnd-sim file from the *same set of events*
FILE_PATHS = {
    "edep":  "/media/disk1/jwolcott/data/scratch/larcv_neutrino.0_1624283861.larcv.npz",
    "larnd": "/media/disk1/jwolcott/data/scratch/neutrino.0_1624283861.larndsim.larcv.reco.npz"
}

In [ ]:
REQUIRED_VARS = ["input_data", "event_base"]

In [ ]:
# Load the reco output from the files here
import numpy as np
files =	{ k: np.load(open(f, "rb"), allow_pickle=True) for k, f in FILE_PATHS.items() }

# these are usually dicts, so the actual type needs to be reconstructed
DATA = {}
for filetype, datafile in files.items():
    for k in datafile:
        if k not in REQUIRED_VARS:
            continue

        if k not in DATA:
            DATA[k] = {}

        print("Loading key:", k, type(datafile[k]))
        try:
            DATA[k][filetype] = datafile[k].item()
        except:
            DATA[k][filetype] = datafile[k]


In [ ]:
# now find the events that match between the two files  (usually some don't due to suppression of events with no hits)

event_ids = { k: set(np.unique([evb[0][2] for evb in DATA["event_base"][k]])) for k in FILE_PATHS }
common_evts = set.intersection(*event_ids.values())

keep = {}
for filetype, product in DATA["event_base"].items():
    keep[filetype] = [idx for idx in range(len(product)) if product[idx][0][2] in common_evts]
    print("for sample", filetype, "keep", len(keep[filetype]), "events")#, keep[filetype])

print("edep-sim:", keep["edep"])
print("larnd-sim:", keep["larnd"])

MATCHED_DATA = {}
for k in DATA:
    MATCHED_DATA[k] = {}
    for filetype, product in DATA[k].items():
        MATCHED_DATA[k][filetype] = [v for i, v in enumerate(DATA[k][filetype]) if i in keep[filetype]]

In [ ]:
# plot some sanity checks on nhits to make sure it looks like the events actually match

from matplotlib import pyplot as plt

plt.figure(figsize=(8, 6), facecolor="white")

#print(MATCHED_DATA["input_data"]["larnd"])
nhits = { k: [len(ev) for ev in vals] for k, vals in MATCHED_DATA["input_data"].items() }

for datatype, hitlist in nhits.items():
    plt.plot(range(len(hitlist)), hitlist, label=datatype)
plt.xlabel("Event idx after cut")
plt.ylabel("$N_{hits}$")
plt.semilogy()
plt.legend()
plt.show()

plt.figure(facecolor="white")
for datatype, hitlist in nhits.items():
    plt.hist(hitlist, histtype='step', label=datatype, bins=np.logspace(0, 5, 50))
plt.xlabel("$N_{hits}$")
plt.xlabel("Events")
plt.semilogx()
plt.legend()
plt.show()



In [ ]:
# set up for making the histogram we need

from plotting_helpers import hist_aggregate
from utility_functions import find_matching_rows


@hist_aggregate("mev_vs_adc",
                hist_dim=2,
                bins=(np.logspace(4.7, 5.5, 100),
                      np.logspace(-3, 0.2, 100)) )
def agg_adc_vs_trueedep(vals):
    """    """
    edep_pos = np.array(vals["edep"][:, :3])  # need to copy so array is contiguous and can be rearranged in find_matching_rows() below
    larnd_pos = np.array(vals["larnd"][:, :3]) # ditto

#    print('edep_pos:', edep_pos.flags, edep_pos)
#    print('larnd_pos:', larnd_pos.flags, larnd_pos)
    larnd_match_idxs = find_matching_rows(larnd_pos, edep_pos)

    larnd_electrons = []
    edep_energy = []
    edep_match_idxs = []
    for idx in larnd_match_idxs[0]:
        # print("considering larnd idx:", idx)
        # print("larnd row:", larnd_pos[idx, None, :])
        match_rows = find_matching_rows(edep_pos, larnd_pos[idx, None, :])  # the extra 'None' results in a 2D array, which we need for the find_matching_rows(), rather than a 1D one
        if len(match_rows) > 1:
            print("too many matches for vox at:", larnd_pos[idx])
            continue
        elif len(match_rows) == 0:
            print("coudn't find edep-sim row for allegedly matched larnd-sim row:", larnd_pos[idx])
            continue
        edep_match_idxs.append(match_rows)

        # print("matched:")
        # print("  edep:", vals["edep"][match_rows])
        # print("  larnd:", vals["larnd"][idx])
#        print(vals["edep"][match_rows, 4], type(vals["edep"][match_rows, 4]))
        edep_energy.append(float(vals["edep"][match_rows, 4]))  # type cast because otherwise it's a numpy array [[val]]
        larnd_electrons.append(float(vals["larnd"][idx, 4]))

    unselected_edep_rows = np.delete(vals["edep"], edep_match_idxs, axis=0)
    unselected_larnd_rows = np.delete(vals["larnd"], larnd_match_idxs, axis=0)
    print(len(unselected_edep_rows), "unmatched edep-sim hits (", len(vals["edep"]), "total)")
    print("   10 most energetic:")
    print(unselected_edep_rows[unselected_edep_rows[:, 4].argsort()][-10:])
    print(len(unselected_larnd_rows), "unmatched larnd-sim hits (", len(vals["larnd"]), "total)")
    print("   10 most energetic:")
    print(unselected_larnd_rows[unselected_larnd_rows[:, 4].argsort()][-10:])
    # print("returning (electrons, energy) entries of lengths", (len(larnd_electrons), len(edep_energy)))
    # print("larnd_electrons[:20]:", larnd_electrons[:20])
    # print("edep_energy[:20]:", edep_energy[:20])

    return (larnd_electrons, edep_energy)

In [ ]:
# more histogramming infrastructure

import plotting_helpers
@plotting_helpers.req_vars_hist(["input_data", "event_base"])
def BuildHists(data, hists):
    n_evt_by_ds = {dsname: len(d) for dsname, d in data["input_data"].items()}
    assert len(set(n_evt_by_ds.values())) == 1, "Data sets do not have the same number of events: %s" % n_evt_by_ds
    n_evts = list(n_evt_by_ds.values())[0]
    print('n_evts:', n_evt_by_ds)

    for evt_idx in range(n_evts):
        for agg_fn in (agg_adc_vs_trueedep,):
            agg_fn({dsname: d[evt_idx] for dsname, d in data["input_data"].items()}, hists)


In [ ]:
# more histogramming infrastructure

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import colors

def PlotHists(hists, outdir=None, fmts=None):
    h = hists["mev_vs_adc"]
    fig = plt.figure(facecolor="white")
    ax = fig.add_subplot()
    x, y = np.meshgrid(*h.bins)
    im = ax.pcolormesh(x, y, h.data.T, cmap="Reds", norm=colors.LogNorm(vmin=1e-1, vmax=h.data.T.max()))
    plt.colorbar(im)

    ax.set_xlabel(r"$N_{e^{-}}$ at anode (larnd-sim)")
    ax.set_ylabel("True energy deposited in voxel (edep-sim) (MeV)")

    if not any(x is None for x in (outdir, fmts)):
        plotting_helpers.savefig(fig, "trueedep-vs-Nelec", outdir, fmts)

In [ ]:
# now go!

hists = {}
BuildHists(MATCHED_DATA, hists)


In [ ]:
# make the plots

PlotHists(hists, outdir="/tmp")
